In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

from pathlib import Path

import keras_tuner as kt

from sklearn.model_selection import train_test_split

from importlib import reload
import hyper_model_creator

In [2]:
trials_dir = Path('trials', 'hyperband_resnet_followup')
trial_metric = 'val_weightedf2score'

In [ ]:
reload(hyper_model_creator)

hypermodel = hyper_model_creator.BuildHyperModel(
    trials_dir, trial_metric
)

tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective(trial_metric, direction='max'),
    directory=trials_dir.parent,
    project_name=trials_dir.name,
    hyperband_iterations=3,
    max_epochs=30,
    overwrite=False,
)

tuner.search(verbose=1)

Trial 4 Complete [00h 04m 33s]
val_weightedf2score: 0.4774032235145569

Best val_weightedf2score So Far: 0.5666080117225647
Total elapsed time: 00h 20m 13s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
32                |64                |filter_size_min
512               |256               |filter_size_max
4                 |4                 |convolutional_layers
64                |256               |units_min
2048              |1024              |units_max
4                 |4                 |dense_layers
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 154s 38ms/step - accuracy: 0.2137 - auc: 0.5931 - loss: 0.6605 - prc: 0.3091 - precision: 0.3395 - recall: 0.2492 - weightedf1score: 0.2968 - weightedf2score: 0.2648 - val_accuracy: 0.3755 - val_au

In [ ]:
# %load_ext tensorboard
# tb_path = str(trials_dir.joinpath('board'))
# %tensorboard --logdir "$tb_path"